Autor: Natalia Kiełbasa

In [1]:
import numpy as np

## Opis projektu - Sweep parametrów środowiska w części A1.4:

### Co już mamy w pliku z rodziałem 3:
Tutaj mamy bazując na A1.3 i A1.2 wyliczenie optymalnych polityk w zależności od zmieniających się parametrów środowiska. Jest to świetny przykład tego, że w trakcie jak zmienia nam się nasze otoczenie/środowisko, to nasza polityka (sposób działania) może przestać być optymalna. Tutaj (A1.4) mamy na bardzo prostym przykładzie - naszym modelu odkurzacza, który mam nadzieję już w miarę wszyscy rozumiemy, pokazane jak to się zmienia. 
W aktualnym przykładzie zmieniamy tylko beta dla 6 wartości oraz rescue_cost dla 4 wartości:

- beta_list = [0.1, 0.3, 0.5, 0.7, 0.9]
- rescue_list = [-1.0, -3.0, -6.0, -10.0]

Następnie iterujemy po nich i sprawdzamy dla każdej kombinacji optymalną politykę:

- Ustawienia stałe: alpha= 0.8 r_search= 5.0 r_wait= 1.0 gamma= 0.9 
- Obliczenia: beta rescue_cost pi(H) pi(L): 0.1 -1.0 SEARCH RECHARGE0.1 -3.0 SEARCH RECHARGE0.1 -6.0 SEARCH RECHARGE0.1 -10.0 SEARCH RECHARGE0.3 -1.0 SEARCH RECHARGE0.3 -3.0 SEARCH RECHARGE0.3 -6.0 SEARCH RECHARGE0.3 -10.0 SEARCH RECHARGE0.5 -1.0 SEARCH SEARCH 0.5 -3.0 SEARCH RECHARGE0.5 -6.0 SEARCH RECHARGE0.5 -10.0 SEARCH RECHARGE0.7 -1.0 SEARCH SEARCH 0.7 -3.0 SEARCH RECHARGE0.7 -6.0 SEARCH RECHARGE0.7 -10.0 SEARCH RECHARGE0.9 -1.0 SEARCH SEARCH 0.9 -3.0 SEARCH SEARCH 0.9 -6.0 SEARCH SEARCH 0.9 -10.0 SEARCH SEARCH Wskazówka: obserwuj, kiedy π*(L) zmienia się z SEARCH na RECHARGE.

Widać na przykład, że nasz najprostszy działający i logiczny model - rozładowane to ładujemy, naładowane to odkurzamy nie zawsze jest optymalny. Da się to pewnie wytłumaczyć, np. skrajne sytuacje takie jak skrajnie zużyta bateria i szybko rozładowująca się (znamy z laptopów :).

### Zadanie:
W zadaniu chodzi o to, aby w tak prostym przykładzie pobawić się parametrami środowiska (alpha, beta, rescue_cost, gamma, r_wait, r_search) i zobaczyć jak nimi manipulując możemy uzyskiwać różne kombinacje optymanych polityk.
1. Wykonać ręcznie w zadaniu A1.2 ewaluację kilku ręcznie wstawionych polityk i zobaczyć, które będą najlepsze. Pogrubione możemy zmieniać  sobie na inne możliwe akcje, aby zobaczyć co daje najlepszy wynik.
- Polityka 1: w H -> SEARCH, w L -> RECHARGE
- pi1 = np.zeros((nS, nA))
- pi1[H, SEARCH] = 1.0
- pi1[L, RECHARGE] = 1.0
2. Następnie można przejść do A1.3, gdzie jest główny algorytm znajdujący optymalną politykę przy zadanych parametrach.
3. (główna część zadania) W części A1.4 wykonujemy "sweepy" po różnych zestawach parametrów środowiska, oczywiście nie po wszystkich, tylklo proszę wybrać kilka i spróbować ocenić z czego to wynika/zinterepretować.
4. Ważne! Trzeba jasno napisać:
- co jest częścią środowiska
- co jest częścią agenta
- jaką mamy politykę
- jak wyliczamy funkcję wartości V_{pi}
- jak ewaluujemy politykę

### A1.2 - ewaluacja własnych polityk

- polityka: czyli reguła która mówi agentowi jakie działania może podjąć w danym stanie
- $v_\pi$: wartość oczekiwanej nagrody w danym stanie

In [2]:
def evaluate_policy_linear_system(P_pi: np.ndarray, r_pi: np.ndarray, gamma: float) -> np.ndarray:
    """Rozwiązuje równanie Bellmana dla danej polityki w postaci macierzowej.

    Dla polityki pi mamy:
        v = r_pi + gamma * P_pi * v
    czyli:
        (I - gamma * P_pi) v = r_pi

    Zwraca wektor v (shape: [nS]).
    """
    nS = P_pi.shape[0]
    I = np.eye(nS)
    return np.linalg.solve(I - gamma * P_pi, r_pi)

def pretty_matrix_as_grid(v: np.ndarray, nrow: int, ncol: int, decimals: int = 1):
    """Pomocniczo: wyświetl wektor wartości jako siatkę (nrow x ncol)."""
    grid = v.reshape(nrow, ncol)
    with np.printoptions(precision=decimals, suppress=True):
        print(grid)

def action_arrows(pi_det: np.ndarray, nrow: int, ncol: int):
    """Zamienia deterministyczną politykę (akcja w każdym stanie) na strzałki w siatce."""
    arrows = {0:'↑', 1:'→', 2:'↓', 3:'←', None:'·'}
    out = []
    for r in range(nrow):
        row = []
        for c in range(ncol):
            s = r*ncol + c
            a = int(pi_det[s]) if pi_det[s] is not None else None
            row.append(arrows.get(a, '?'))
        out.append(' '.join(row))
    print('\n'.join(out))

In [3]:
# Budowa P_pi oraz r_pi (rozwiązanie)
def build_P_r_for_policy(P, pi):
    """
    Buduje (P_pi, r_pi) dla zadanej polityki π.

    Wejście:
    - P: model środowiska w formacie
         P[s][a] -> lista (p, s2, r, terminated)
    - pi: polityka stochastyczna w postaci macierzy (nS, nA),
          pi[s, a] = P(A_t = a | S_t = s)

    Wyjście:
    - P_pi: macierz przejść dla polityki π (nS x nS)
    - r_pi: wektor nagród oczekiwanych dla polityki π (nS,)

    Sens:
    Z ogólnego MDP (P[s][a]) robimy „świat widziany przez politykę π”,
    potrzebny do rozwiązania równania:
        v = r_pi + γ P_pi v
    """
    nS = len(P)              # liczba stanów
    nA = len(P[0])           # liczba akcji (z pierwszego stanu)

    # Macierz przejść i wektor nagród dla polityki π
    P_pi = np.zeros((nS, nS), dtype=float)
    r_pi = np.zeros(nS, dtype=float)

    # Iterujemy po wszystkich stanach
    for s in range(nS):

        # Iterujemy po wszystkich akcjach
        for a in range(nA):

            # Waga akcji a w stanie s wg polityki π
            w = float(pi[s, a])

            # Jeśli polityka nigdy nie wybiera tej akcji, pomijamy ją
            if w == 0.0:
                continue

            outcomes = P[s][a]

            # Jeśli akcja jest niedostępna (pusta lista), pomijamy
            if not outcomes:
                continue

            # Iterujemy po wszystkich możliwych skutkach akcji a w stanie s
            for (p, s2, r, terminated) in outcomes:

                # Składnik nagrody oczekiwanej:
                # r_pi[s] = E_π[R_{t+1} | S_t = s]
                r_pi[s] += w * p * float(r)

                # Składnik przejść:
                # P_pi[s, s2] = P(S_{t+1} = s2 | S_t = s, π)
                #
                # Jeśli przejście jest terminalne:
                # - V(s2) = 0 w równaniu Bellmana,
                # - więc NIE dodajemy go do macierzy P_pi
                if not terminated:
                    P_pi[s, int(s2)] += w * p

    return P_pi, r_pi


In [4]:
# Robot zdefiniowany na zajęciach. Ma 2 możliwe stany i może wykonać 3 akcje
def build_recycling_robot_P(alpha=0.8, beta=0.4, r_search=5.0, r_wait=1.0, rescue_cost=-3.0):
    """Return (P, nS, nA) for Recycling Robot.

    States: 0=H, 1=L
    Actions: 0=SEARCH, 1=WAIT, 2=RECHARGE (only in L)

    P[s][a] -> list (p, s2, r, terminated)
    """
    # 1) Rozmiar MDP: 2 stany (H,L) i 3 akcje (SEARCH, WAIT, RECHARGE)
    nS, nA = 2, 3
    # 2) Inicjalizacja pustej struktury przejść
    #    P[s][a] = [] oznacza brak zdefiniowanych przejść (np. akcja niedostępna)
    P = {s: {a: [] for a in range(nA)} for s in range(nS)}
    # 3) Aliasowanie indeksów dla czytelności

    H, L = 0, 1
    SEARCH, WAIT, RECHARGE = 0, 1, 2
    # -----------------------------
    # Stan H (wysoka energia)
    # -----------------------------
    # SEARCH w H: nagroda r_search; przejście do H z p=alpha, do L z p=1-alpha
    
    # High energy (H)
    P[H][SEARCH] = [
        (alpha, H, r_search, False),
        (1 - alpha, L, r_search, False),
    ]
    # WAIT w H: zawsze zostajemy w H; nagroda r_wait
    P[H][WAIT] = [
        (1.0, H, r_wait, False),
    ]
    # RECHARGE w H: akcja niedostępna w tym stanie
    P[H][RECHARGE] = []  # not available in H

    # -----------------------------
    # Stan L (niska energia)
    # -----------------------------
    # SEARCH w L:
    #   - z p=beta: zostajemy w L i dostajemy r_search
    #   - z p=1-beta: rozładowanie -> powrót do H i kara rescue_cost
    # Low energy (L)
    P[L][SEARCH] = [
        (beta, L, r_search, False),
        (1 - beta, H, rescue_cost, False),
    ]
    # WAIT w L: zawsze zostajemy w L; nagroda r_wait
    P[L][WAIT] = [
        (1.0, L, r_wait, False),
    ]
    # RECHARGE w L: zawsze przechodzimy do H; nagroda 0
    P[L][RECHARGE] = [
        (1.0, H, 0.0, False),
    ]

    return P, nS, nA

# --------------------------------------------------
# Testy poprawności modelu
# --------------------------------------------------

# --- testy ---
P, nS, nA = build_recycling_robot_P(alpha=0.8, beta=0.4, r_search=5.0, r_wait=1.0, rescue_cost=-3.0)

assert nS == 2 and nA == 3
assert abs(sum(p for p, *_ in P[0][0]) - 1.0) < 1e-12  # H, SEARCH
assert abs(sum(p for p, *_ in P[0][1]) - 1.0) < 1e-12  # H, WAIT
assert P[0][2] == []                                  # H, RECHARGE not allowed
assert abs(sum(p for p, *_ in P[1][0]) - 1.0) < 1e-12  # L, SEARCH
assert abs(sum(p for p, *_ in P[1][1]) - 1.0) < 1e-12  # L, WAIT
assert abs(sum(p for p, *_ in P[1][2]) - 1.0) < 1e-12  # L, RECHARGE

print("A1.1 tests passed ✅")


A1.1 tests passed ✅


Ręczne wypisanie przykładowych polityk i ich ewaluacja:

In [ ]:
gamma = 0.9
P, nS, nA = build_recycling_robot_P()

H, L = 0, 1
SEARCH, WAIT, RECHARGE = 0, 1, 2


# POLITYKI PODANE NA ZAJĘCIACH
# Polityka 1: w H -> SEARCH, w L -> RECHARGE
pi1 = np.zeros((nS, nA))
pi1[H, SEARCH] = 1.0
pi1[L, RECHARGE] = 1.0

# Polityka 2: w H -> WAIT, w L -> WAIT
pi2 = np.zeros((nS, nA))
pi2[H, RECHARGE] = 1.0
pi2[L, SEARCH] = 1.0


P_pi1, r_pi1 = build_P_r_for_policy(P, pi1)
v1 = evaluate_policy_linear_system(P_pi1, r_pi1, gamma)

P_pi2, r_pi2 = build_P_r_for_policy(P, pi2)
v2 = evaluate_policy_linear_system(P_pi2, r_pi2, gamma)

print("v_pi1(H), v_pi1(L) =", np.round(v1, 4))
print("v_pi2(H), v_pi2(L) =", np.round(v2, 4))


# WŁASNE PRZYKŁADOWE POLITYKI
# Polityka 3:
pi3 = np.zeros((nS, nA))
pi3[H, WAIT] = 1.0
pi3[L, RECHARGE] = 1.0

# Polityka 4:
pi4 = np.zeros((nS, nA))
pi4[H, SEARCH] = 1.0
pi4[L, SEARCH] = 1.0

# Polityka 5:
pi5 = np.zeros((nS, nA))
pi5[H, SEARCH] = 1.0
pi5[L, WAIT] = 1.0

# Polityka 6:
pi6 = np.zeros((nS, nA))
pi6[H, WAIT] = 1.0
pi6[L, SEARCH] = 1.0


P_pi3, r_pi3 = build_P_r_for_policy(P, pi3)
v3 = evaluate_policy_linear_system(P_pi3, r_pi3, gamma)

P_pi4, r_pi4 = build_P_r_for_policy(P, pi4)
v4 = evaluate_policy_linear_system(P_pi4, r_pi4, gamma)

P_pi5, r_pi5 = build_P_r_for_policy(P, pi5)
v5 = evaluate_policy_linear_system(P_pi5, r_pi5, gamma)

P_pi6, r_pi6 = build_P_r_for_policy(P, pi6)
v6 = evaluate_policy_linear_system(P_pi6, r_pi6, gamma)

print("v_pi3(H), v_pi3(L) =", np.round(v3, 4))
print("v_pi4(H), v_pi4(L) =", np.round(v4, 4))
print("v_pi5(H), v_pi5(L) =", np.round(v5, 4))
print("v_pi6(H), v_pi6(L) =", np.round(v6, 4))


v_pi1(H), v_pi1(L) = [42.3729 38.1356]
v_pi2(H), v_pi2(L) = [0.     0.3125]
v_pi3(H), v_pi3(L) = [10.  9.]
v_pi4(H), v_pi4(L) = [39.4634 33.6098]
v_pi5(H), v_pi5(L) = [24.2857 10.    ]
v_pi6(H), v_pi6(L) = [10.    8.75]


### Wnioski z tej ewaluacji:
- polityka 1 daje największą oczekiwaną sumę nagród
- z własnych polityk polityka 4 też dała wysoką sumę nagród. Była to politykka która zakładała szukanie w każdym stanie (bardzo agresywna)
- ogólnie bardzo bezpieczne polityki, które głównie czekają mają kiepskie wyniki, co ma sens patrząc na to że nie prowadzą nas do przodu

### A1.3: przenosimy tu funkcje z oryginalnego zadania z ćwiczeń

In [6]:
def all_deterministic_policies_robot():
    """
    Generuje wszystkie deterministyczne polityki dla robota recyklingowego.

    Ponieważ:
    - w stanie H dostępne są 2 sensowne akcje (SEARCH, WAIT),
    - w stanie L dostępne są 3 akcje (SEARCH, WAIT, RECHARGE),

    liczba wszystkich deterministycznych polityk wynosi 2 * 3 = 6.

    Każda polityka pi jest macierzą:
        pi[s, a] = 1  -> akcja a jest zawsze wybierana w stanie s
        pi[s, a] = 0  -> akcja a nigdy nie jest wybierana w stanie s
    """
    # Indeksy stanów
    H, L = 0, 1

    # Indeksy akcji
    SEARCH, WAIT, RECHARGE = 0, 1, 2

    policies = []

    # Wybieramy wszystkie możliwe akcje w stanie H
    for aH in [SEARCH, WAIT]:

        # Wybieramy wszystkie możliwe akcje w stanie L
        for aL in [SEARCH, WAIT, RECHARGE]:

            # Tworzymy pustą politykę (2 stany x 3 akcje)
            pi = np.zeros((2, 3))

            # Polityka deterministyczna:
            # w stanie H zawsze wybieramy akcję aH
            pi[H, aH] = 1.0

            # w stanie L zawsze wybieramy akcję aL
            pi[L, aL] = 1.0

            # Dodajemy gotową politykę do listy
            policies.append(pi)

    return policies


def best_policy_robot(P, gamma=0.9):
    """
    Wybiera najlepszą politykę spośród wszystkich deterministycznych polityk.

    Kryterium:
    - maksymalizujemy wartość v_pi(H),
      czyli oczekiwany zdyskontowany zwrot,
      jeśli startujemy w stanie H i stosujemy politykę pi.

    Uwaga dydaktyczna:
    - to jest brute force (sprawdzamy wszystkie polityki),
    - działa tylko dlatego, że MDP jest bardzo małe.
    """
    best_pi = None    # najlepsza znaleziona polityka
    best_vH = None    # jej wartość v_pi(H)

    # Iterujemy po wszystkich możliwych politykach
    for pi in all_deterministic_policies_robot():

        # Budujemy model świata "widoczny" dla danej polityki:
        # P_pi   — macierz przejść dla tej polityki
        # r_pi   — wektor nagród dla tej polityki
        P_pi, r_pi = build_P_r_for_policy(P, pi)

        # Liczymy dokładnie funkcję wartości v_pi,
        # rozwiązując układ równań Bellmana:
        # v = r_pi + gamma * P_pi * v
        v = evaluate_policy_linear_system(P_pi, r_pi, gamma)

        # Interesuje nas wartość w stanie H
        vH = float(v[0])

        # Sprawdzamy, czy ta polityka jest lepsza od dotychczasowej
        if best_vH is None or vH > best_vH:
            best_vH = vH
            best_pi = pi

    return best_pi, best_vH


# --------------------------------------------------
# Uruchomienie: znalezienie polityki optymalnej
# --------------------------------------------------

# Budujemy model świata robota
P, nS, nA = build_recycling_robot_P()

# Szukamy najlepszej polityki (brute force)
pi_star, vH_star = best_policy_robot(P, gamma=0.9)

# Mapowanie indeksów akcji na czytelne nazwy
action_name = {0: "SEARCH", 1: "WAIT", 2: "RECHARGE"}

print("Najlepsza polityka (wg v(H)) ma v(H) =", round(vH_star, 4))
print("pi*(H) =", action_name[int(np.argmax(pi_star[0]))])
print("pi*(L) =", action_name[int(np.argmax(pi_star[1]))])


Najlepsza polityka (wg v(H)) ma v(H) = 42.3729
pi*(H) = SEARCH
pi*(L) = RECHARGE


Czyli wcześniejsza polityka 1 faktycznie była najlepsza z deterministycznych

### A1.4: wykonujemy sweepy po różnych zwestawach parametrów

Zmieniamy parametray środowiska (alpha, beta, rescue_cost, gamma, r_wait, r_search) i patrzymy jak nimi manipulując możemy uzyskiwać różne kombinacje optymanych polityk. Potem próbujemy ocenić z czego to wynika/zinterepretować.

Uwzględniamy:
- co jest częścią środowiska
- co jest częścią agenta
- jaką mamy politykę
- jak wyliczamy funkcję wartości V_{pi}
- jak ewaluujemy politykę

1. Co jest częścią środowiska:  wszystko, co działa niezależnie od decyzji agenta
    - alpha - prawdopodobieństwo, że SEARCH w stanie H zakończy się sukcesem
    - beta - prawdopodobieństwo, że SEARCH w stanie L zakończy się sukcesem
    - rescue_cost - kara za przejście do stanu utknięcia
    - r_search, r_wait - nagrody za akcje

2. Co jest częścią agenta: 
    - gamma - współczynnik dyskontowania, czyli jak bardzo agent "ceni" przyszłość vs teraźniejszość (to jego perspektywa czasowa)
    - polityka - strategia wyboru akcji (patrzymy jak różne parametry mają wpływ na optymalne polityki)

3. Polityka jest wybierana przez best_policy_robot() (zwraca najlepszą politykę dla danych parametrów)

4. Funkcja wartości jest obliczana przez best_policy_robot()

5. Politykę ewaluujemy jak napisano w zadaniu: maksymalizując wartość v_pi(H), czyli oczekiwany zdyskontowany zwrot, jeśli startujemy w stanie H i stosujemy politykę pi. Jest to jest brute force

In [7]:
# A1.4 — Sweep parametrów: kiedy zmienia się π*(L)
action_name = {0: "SEARCH", 1: "WAIT", 2: "RECHARGE"}  # mapowanie akcji

# Wartości domyślne zgodne z build_recycling_robot_P() i best_policy_robot()
alpha_default = 0.8
beta_default = 0.4
r_search_default = 5.0
r_wait_default = 1.0
rescue_cost_default = -3.0
gamma_default = 0.9

# Wartości eksperymentalne
alpha_list = [0.1, 0.3, 0.5, 0.7, 0.9]
beta_list = [0.1, 0.3, 0.5, 0.7, 0.9]
r_search_list = [0.5, 2.0, 5.0, 10.0]
r_wait_list= [0.0, 1.0, 3.0, 5.0]
rescue_list = [-1.0, -3.0, -6.0, -10.0, -20.0]
gamma_list = [0.2, 0.5, 0.7, 0.85, 0.9, 0.95, 0.99]

In [8]:
# EKSPERYMENT 1: beta vs rescue_cost, czyli to co było na zajęciach

print("Ustawienia stałe:", "alpha=", alpha_default, "r_search=", r_search_default,
      "r_wait=", r_wait_default, "gamma=", gamma_default)
print("beta   rescue_cost   pi*(H)      pi*(L)")
print("----   ----------    --------    --------")

# Dla każdej pary (beta, rescue_cost) sprawdzamy, jaka polityka jest optymalna
for beta in beta_list:
    for rescue_cost in rescue_list:

        # budujemy świat (MDP) z tymi parametrami
        P, _, _ = build_recycling_robot_P(beta=beta, rescue_cost=rescue_cost)

        # znajdujemy politykę optymalną π* (brute force)
        pi_star, _ = best_policy_robot(P)

        # odczytujemy decyzje w H i L
        aH = action_name[int(np.argmax(pi_star[0]))]
        aL = action_name[int(np.argmax(pi_star[1]))]

        # wypisujemy wynik
        print(f"{beta:0.1f}     {rescue_cost:>10.1f}    {aH:<8}   {aL:<8}")

Ustawienia stałe: alpha= 0.8 r_search= 5.0 r_wait= 1.0 gamma= 0.9
beta   rescue_cost   pi*(H)      pi*(L)
----   ----------    --------    --------
0.1           -1.0    SEARCH     RECHARGE
0.1           -3.0    SEARCH     RECHARGE
0.1           -6.0    SEARCH     RECHARGE
0.1          -10.0    SEARCH     RECHARGE
0.1          -20.0    SEARCH     RECHARGE
0.3           -1.0    SEARCH     RECHARGE
0.3           -3.0    SEARCH     RECHARGE
0.3           -6.0    SEARCH     RECHARGE
0.3          -10.0    SEARCH     RECHARGE
0.3          -20.0    SEARCH     RECHARGE
0.5           -1.0    SEARCH     SEARCH  
0.5           -3.0    SEARCH     RECHARGE
0.5           -6.0    SEARCH     RECHARGE
0.5          -10.0    SEARCH     RECHARGE
0.5          -20.0    SEARCH     RECHARGE
0.7           -1.0    SEARCH     SEARCH  
0.7           -3.0    SEARCH     RECHARGE
0.7           -6.0    SEARCH     RECHARGE
0.7          -10.0    SEARCH     RECHARGE
0.7          -20.0    SEARCH     RECHARGE
0.9         

### Obserwacje:
- polityka dla H nie zmienia się (zawsze SEARCH), więc wydaje się stabilna
- przy niskim beta dla L agent nie wybiera search co ma sens patrząć na to że czym mniejsze beta, tym mniejsza szansa, że search w L zakończy się sukcesem
- tak samo dla zwiększanie ujemnego rescue_cost powoduje przejście w L z search do recharge, wysoka kara za awarię zniechęca do bartdziej ryzykownego search w stanie L

In [9]:
# EKSPERYMENT 2: alfa vs rescue_cost

print("Ustawienia stałe:", "beta=", beta_default, "r_search=", r_search_default,
      "r_wait=", r_wait_default, "gamma=", gamma_default)
print("alpha   rescue_cost   pi*(H)      pi*(L)")
print("----   ----------    --------    --------")

for alpha in alpha_list:
    for rescue_cost in rescue_list:

        P, _, _ = build_recycling_robot_P(alpha=alpha, rescue_cost=rescue_cost)
        pi_star, _ = best_policy_robot(P)

        aH = action_name[int(np.argmax(pi_star[0]))]
        aL = action_name[int(np.argmax(pi_star[1]))]

        print(f"{alpha:0.1f}     {rescue_cost:>10.1f}    {aH:<8}   {aL:<8}")

Ustawienia stałe: beta= 0.4 r_search= 5.0 r_wait= 1.0 gamma= 0.9
alpha   rescue_cost   pi*(H)      pi*(L)
----   ----------    --------    --------
0.1           -1.0    SEARCH     SEARCH  
0.1           -3.0    SEARCH     RECHARGE
0.1           -6.0    SEARCH     RECHARGE
0.1          -10.0    SEARCH     RECHARGE
0.1          -20.0    SEARCH     RECHARGE
0.3           -1.0    SEARCH     SEARCH  
0.3           -3.0    SEARCH     RECHARGE
0.3           -6.0    SEARCH     RECHARGE
0.3          -10.0    SEARCH     RECHARGE
0.3          -20.0    SEARCH     RECHARGE
0.5           -1.0    SEARCH     SEARCH  
0.5           -3.0    SEARCH     RECHARGE
0.5           -6.0    SEARCH     RECHARGE
0.5          -10.0    SEARCH     RECHARGE
0.5          -20.0    SEARCH     RECHARGE
0.7           -1.0    SEARCH     RECHARGE
0.7           -3.0    SEARCH     RECHARGE
0.7           -6.0    SEARCH     RECHARGE
0.7          -10.0    SEARCH     RECHARGE
0.7          -20.0    SEARCH     RECHARGE
0.9         

### Obserwacje:
- stan H jest taki sam niezależnie od parametrów alpha i rescue cost, tak samo jak w poprzednim przykładzie. Chociaż tu jest to na tyle ciekawe że nawet niskie wartości alpha tego nie zmieniają
- dla dużego alpha agent mocno preferuje recharge w stanie L, co może wskazywać na to że szybko się kończy energia potrzebna do zzostania w stanie H, więc częstsze recharge są potrzebne

In [10]:
# EKSPERYMENT 3: Wpływ alpha i beta

print("Ustawienia stałe:", "rescue_cost=", rescue_cost_default, "r_search=", r_search_default,
      "r_wait=", r_wait_default, "gamma=", gamma_default)
print("alpha   beta         pi*(H)      pi*(L)")
print("----   ----------    --------    --------")

for alpha in alpha_list:
    for beta in beta_list:

        P, _, _ = build_recycling_robot_P(alpha=alpha, beta=beta)
        pi_star, _ = best_policy_robot(P)

        aH = action_name[int(np.argmax(pi_star[0]))]
        aL = action_name[int(np.argmax(pi_star[1]))]

        print(f"{alpha:0.1f}     {beta:>10.1f}    {aH:<8}   {aL:<8}")

Ustawienia stałe: rescue_cost= -3.0 r_search= 5.0 r_wait= 1.0 gamma= 0.9
alpha   beta         pi*(H)      pi*(L)
----   ----------    --------    --------
0.1            0.1    SEARCH     RECHARGE
0.1            0.3    SEARCH     RECHARGE
0.1            0.5    SEARCH     RECHARGE
0.1            0.7    SEARCH     SEARCH  
0.1            0.9    SEARCH     SEARCH  
0.3            0.1    SEARCH     RECHARGE
0.3            0.3    SEARCH     RECHARGE
0.3            0.5    SEARCH     RECHARGE
0.3            0.7    SEARCH     SEARCH  
0.3            0.9    SEARCH     SEARCH  
0.5            0.1    SEARCH     RECHARGE
0.5            0.3    SEARCH     RECHARGE
0.5            0.5    SEARCH     RECHARGE
0.5            0.7    SEARCH     SEARCH  
0.5            0.9    SEARCH     SEARCH  
0.7            0.1    SEARCH     RECHARGE
0.7            0.3    SEARCH     RECHARGE
0.7            0.5    SEARCH     RECHARGE
0.7            0.7    SEARCH     SEARCH  
0.7            0.9    SEARCH     SEARCH  
0.9  

### Obserwacje:
- nadal dominuje search w H
- duże beta (> 0.7) sprzyja polityce search w stanie L, pokazuje to że dopiero wtedy ryzyko szukania bez recharge wydaje się warte, inaczej recharge jest preferowane
- ogólnie widać że głównie beta wpływa na wybór search w L, co ma sens patrząc na to że beta pokazuje szansę sukcesu szukania w L, a alpha zzamiast tego w H

In [11]:
# EKSPERYMENT 4: Wpływ nagród r_search i r_wait za akcje

print("Ustawienia stałe:", "rescue_cost=", rescue_cost_default, "alpha=", alpha_default,
      "beta=", beta_default, "gamma=", gamma_default)
print("r_search r_wait         pi*(H)      pi*(L)")
print("----    ----------    --------    --------")

for r_search in r_search_list:
    for r_wait in r_wait_list:

        P, _, _ = build_recycling_robot_P(r_search=r_search, r_wait=r_wait)
        pi_star, _ = best_policy_robot(P)

        aH = action_name[int(np.argmax(pi_star[0]))]
        aL = action_name[int(np.argmax(pi_star[1]))]

        print(f"{r_search:0.1f}     {r_wait:>10.1f}    {aH:<8}   {aL:<8}")

Ustawienia stałe: rescue_cost= -3.0 alpha= 0.8 beta= 0.4 gamma= 0.9
r_search r_wait         pi*(H)      pi*(L)
----    ----------    --------    --------
0.5            0.0    SEARCH     RECHARGE
0.5            1.0    WAIT       SEARCH  
0.5            3.0    WAIT       SEARCH  
0.5            5.0    WAIT       SEARCH  
2.0            0.0    SEARCH     RECHARGE
2.0            1.0    SEARCH     RECHARGE
2.0            3.0    WAIT       SEARCH  
2.0            5.0    WAIT       SEARCH  
5.0            0.0    SEARCH     RECHARGE
5.0            1.0    SEARCH     RECHARGE
5.0            3.0    SEARCH     RECHARGE
5.0            5.0    SEARCH     WAIT    
10.0            0.0    SEARCH     RECHARGE
10.0            1.0    SEARCH     RECHARGE
10.0            3.0    SEARCH     RECHARGE
10.0            5.0    SEARCH     RECHARGE


### Obserwacje:
- w odróżnieniu od wcześniejszych przykładów, widzimy wreszcie zmiany w H, pokazuje to że zmiany nagród mają tutaj większe znaczenie
- całościowo też pierwszy raz pojawiaja się WAIT - dla niskiego r_search w H (bo wtedy szukanie nie jest tak bardzo nagradzane) i z jakiegoś powodu dla r_search i r_wait = 5.0. Myślę że można to podsumować jako - jeśli nagroda zza czekanie jest podobna lub większa niż nagroda za szukanie, zaczyna pojawiać się dominancja stanu wait

In [15]:
# EKSPERYMENT 5: Wpływ kary za awarię i dyskontowania

print("Ustawienia stałe:", "beta=", beta_default, "r_search=", r_search_default,
      "r_wait=", r_wait_default, "alpha=", alpha_default)
print("gamma   rescue_cost   pi*(H)      pi*(L)")
print("----   ----------    --------    --------")

for gamma in gamma_list:
    for rescue_cost in rescue_list:

        P, _, _ = build_recycling_robot_P(rescue_cost=rescue_cost)
        pi_star, _ = best_policy_robot(P, gamma=gamma)

        aH = action_name[int(np.argmax(pi_star[0]))]
        aL = action_name[int(np.argmax(pi_star[1]))]

        print(f"{gamma:0.2f}     {rescue_cost:>10.1f}    {aH:<8}   {aL:<8}")

Ustawienia stałe: beta= 0.4 r_search= 5.0 r_wait= 1.0 alpha= 0.8
gamma   rescue_cost   pi*(H)      pi*(L)
----   ----------    --------    --------
0.20           -1.0    SEARCH     SEARCH  
0.20           -3.0    SEARCH     WAIT    
0.20           -6.0    SEARCH     WAIT    
0.20          -10.0    SEARCH     WAIT    
0.20          -20.0    SEARCH     WAIT    
0.50           -1.0    SEARCH     SEARCH  
0.50           -3.0    SEARCH     RECHARGE
0.50           -6.0    SEARCH     RECHARGE
0.50          -10.0    SEARCH     RECHARGE
0.50          -20.0    SEARCH     RECHARGE
0.70           -1.0    SEARCH     SEARCH  
0.70           -3.0    SEARCH     RECHARGE
0.70           -6.0    SEARCH     RECHARGE
0.70          -10.0    SEARCH     RECHARGE
0.70          -20.0    SEARCH     RECHARGE
0.85           -1.0    SEARCH     RECHARGE
0.85           -3.0    SEARCH     RECHARGE
0.85           -6.0    SEARCH     RECHARGE
0.85          -10.0    SEARCH     RECHARGE
0.85          -20.0    SEARCH     R

### Obserwacje:
- stan H jest znowu zdominowany przezz search
- niskie gamma pokazuje że agent ma dosyć agresyną politykę, co jest spowodowane tym że działa w małym przedziale czasowym i że tak powiem nie przejmuje się tak bardzzo przyszłością (jest ogólnie krótkowzroczny). Unika wtedy recharge bo to bardziej długoterminowa korzyść i czasami w stanie L ucieka się do czekania, pewnie żeby odroczyć decyzję pry wysokim rescue_cost

In [16]:
# EKSPERYMENT 6: Wpływ samego gamma

print("Ustawienia stałe:", "beta=", beta_default, "r_search=", r_search_default,
      "r_wait=", r_wait_default, "alpha=", alpha_default, "rescue_cost=", rescue_cost_default)
print("gamma   pi*(H)      pi*(L)")
print("----    --------    --------")

for gamma in gamma_list:

    P, _, _ = build_recycling_robot_P()
    pi_star, _ = best_policy_robot(P, gamma=gamma)

    aH = action_name[int(np.argmax(pi_star[0]))]
    aL = action_name[int(np.argmax(pi_star[1]))]

    print(f"{gamma:0.2f}    {aH:<8}   {aL:<8}")

Ustawienia stałe: beta= 0.4 r_search= 5.0 r_wait= 1.0 alpha= 0.8 rescue_cost= -3.0
gamma   pi*(H)      pi*(L)
----    --------    --------
0.20    SEARCH     WAIT    
0.50    SEARCH     RECHARGE
0.70    SEARCH     RECHARGE
0.85    SEARCH     RECHARGE
0.90    SEARCH     RECHARGE
0.95    SEARCH     RECHARGE
0.99    SEARCH     RECHARGE


### Obserwacje:
- jak wyżej, zmiany od "standarodwej" polityki search w H i recharge w L widać głównie przy małym gamma. Jest to spowodowane "krótkowzrocznością" agenta przy niskim gamma